In [51]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [52]:
df= pd.read_csv("Boston.csv")

df.head()

,Unnamed: 0,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,black,lstat,medv
0,1,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296,15.3,396.90,4.98,24.0
1,2,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,396.90,9.14,21.6
2,3,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242,17.8,392.83,4.03,34.7
3,4,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222,18.7,394.63,2.94,33.4
4,5,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222,18.7,396.90,5.33,36.2


In [53]:
df= df.drop(df.columns[0], axis=1)
df.head()

,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,black,lstat,medv
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222,18.7,396.90,5.33,36.2


In [54]:
df = df.sample(frac=1, random_state=42).reset_index(drop=True)

In [55]:
X= np.array(df.drop(columns= ['medv']))
y= np.array(df['medv'])

In [56]:
def rescale(x, avg, dev):
    avg= np.mean(x, axis=0)
    dev= np.std(x, axis=0)
    x_rescaled= (x- avg)/dev
    print(x[0])
    x_rescaled[0]
    return x_rescaled

In [57]:
X.shape

(506, 13)

In [68]:
X_train_raw = X[:450]
X_test_raw = X[450:]
train_avg = np.mean(X_train_raw, axis=0)
train_dev = np.std(X_train_raw, axis=0)
X_train = rescale(X_train_raw, train_avg, train_dev)
X_test = rescale(X_test_raw, train_avg, train_dev)
y_train= y[:450]
y_test= y[450:]

[9.1780e-02 0.0000e+00 4.0500e+00 0.0000e+00 5.1000e-01 6.4160e+00
 8.4100e+01 2.6463e+00 5.0000e+00 2.9600e+02 1.6600e+01 3.9550e+02
 9.0400e+00]
[2.3912e-01 0.0000e+00 9.6900e+00 0.0000e+00 5.8500e-01 6.0190e+00
 6.5300e+01 2.4091e+00 6.0000e+00 3.9100e+02 1.9200e+01 3.9690e+02
 1.2920e+01]


In [59]:
def cost(x,y,w,b, l2):
    m= x.shape[0]
    cost=0.0
    for i in range(m):
        cost+= ((np.dot(x[i],w)+b)- y[i])**2
    cost /=(2*m)
    l2_penalty=(l2 / (2 * m)) * np.sum(w**2)
    total= l2_penalty+ cost
    return total

In [60]:
def gradient(x,y,w,b, l2):
    m, n = x.shape
    predictions = np.dot(x, w) + b
    errors = predictions - y
    dw = np.dot(x.T, errors) / m + (l2 / m) * w
    db = np.sum(errors) / m
    return dw, db

In [61]:
def descent(x,y,w,b, alpha, iterations, l1):
    cost_hist=[]
    w_tmp= w;
    b_tmp= b;

    for i in range(iterations):
        dw, db= gradient(x,y,w_tmp, b_tmp, l1)
        w_tmp= w_tmp- alpha* dw
        b_tmp= b_tmp- alpha* db
        if i<1000:
            cost_hist.append(cost(x,y,w_tmp, b_tmp,l1))
        if i%100==0:
            print(f"Iteration {i:4d}: Cost {cost_hist[-1]:8.2f}   ")
    return w_tmp, b_tmp, cost_hist

In [62]:
def r2_score(y_true, y_pred):
    ss_res= np.sum((y_true- y_pred)**2)
    ss_mean= np.sum((y_true- np.mean(y_true))**2)
    r2= 1- (ss_res/ ss_mean)
    return r2

In [63]:
def predict(x, w, b):
    y= np.dot(w,x)+b
    return y

In [64]:
m=X_train[0].shape
w_init= np.zeros(m)
b_init=0
alpha= 0.1
iterations= 1000
l2= 10
w_final, b_final, cost_track= descent(X_train, y_train, w_init, b_init, alpha, iterations, l2)
print(f"b,w found by gradient descent: {b_final:0.2f},{w_final} ")

Iteration    0: Cost   231.76   
Iteration  100: Cost    11.49   
Iteration  200: Cost    11.43   
Iteration  300: Cost    11.42   
Iteration  400: Cost    11.42   
Iteration  500: Cost    11.42   
Iteration  600: Cost    11.42   
Iteration  700: Cost    11.42   
Iteration  800: Cost    11.42   
Iteration  900: Cost    11.42   
b,w found by gradient descent: 22.54,[-0.84706869  0.98665984  0.05992686  0.56418955 -1.77658478  2.741038
  0.15752372 -2.84144369  1.97844343 -1.61545578 -2.00779872  0.85886357
 -3.90873696] 


In [65]:
y_test_pred= predict(X_test.T, w_final, b_final)
r2_test= r2_score(y_test, y_test_pred)
r2_test

np.float64(0.5750400761331673)

In [66]:
y_train_pred= predict(X_train.T, w_final, b_final)
r2_train= r2_score(y_train, y_train_pred)
r2_train

np.float64(0.7499190673276976)

In [67]:
print(w_final)

[-0.84706869  0.98665984  0.05992686  0.56418955 -1.77658478  2.741038
  0.15752372 -2.84144369  1.97844343 -1.61545578 -2.00779872  0.85886357
 -3.90873696]
